<img src="imgs/unicamp.png" width="150" height="150">

## MO444/MC886 - Aprendizado de Máquina e Reconhecimento de Padrões

Esse trabalho foi feito pelos seguintes membros:

- Germán Darío Buitrago Salazar - 164321
- Giovanna Relvas Bartilotti - 023907
- Lucas Zanco Ladeira - 188951
- Rafael Scherer - 204990 

O código original deste projeto está disponível no [repositório do Github](https://github.com/lucaslzl/mo444_p4_qea). 

## Trabalho Final - Serviço de FAQ para os Objetivos de Desenvolvimento Sustentável

<b>Declaração do Problema</b>

"Este trabalho tem como objetivo construir um sistema de Aprendizado de Máquina para solucionar um problema escolhido pelo grupo inserido em um dos Objetivos de Desenvolvimento Sustentável (ODS) da ONU. O trabalho do seu grupo é encontrar uma solução adequada para o problema escolhido."

Neste trabalho foi construído um serviço de FAQ para tirar dúvidas das pessoas sobre os Objetivos de Desenvolvimento Sustentável. Para tal, foram utilizadas algumas estratégias de processamento de linguagem natural, modelos supervisionados, e bibliotecas para auxiliar na construção da base de dados.

## 1. Cenário

De acordo com [1], desenvolvimento sustentável se refere a um modelo de desenvolvimento econômico, social e político que esteja em harmonia com o meio ambiente. Isso significa utilizar conscientemente os recursos disponíveis para que não acabe com o meio ambiente. Ele é imprescindível para o futuro da população humana e dos outros animais que habitam o planeta terra. As Nações Unidas fizeram um acordo em 2015 com 195 países para trabalhar em 17 objetivos de forma a mudar o mundo para melhor. Esses objetivos estão distribuídos em ações para acabar com a pobreza, proteger o meio ambiente e o clima e garantir que as pessoas em todos os lugares possam desfrutar de paz e prosperidade. É possível observar na figura a seguir esses objetivos.

<img src="imgs/ods.png" width="500" height="500">

## 2. Solução

Todos os anos as Nações Unidas publicam um relatório [2] com o avanço dessas ações do ODS. No entanto, esta forma de publicação não é acessível e nem chamativo para a maior parte das pessoas. Sendo assim, é necessário criar uma maneira interativa de levar essas informações para a população. Um serviço de FAQ pode ajudar nesse sentido, com a característica de fazer perguntas abertamente e receber alguma resposta. Para tal, desenvolvemos um serviço que utiliza processamento de linguagem natural (PLN) para identificar sobre qual objetivo o usuário deseja conhecer e qual é a resposta mais provável.

<b>TODO</b>
- Escrever mais sobre PLN
- Descrever melhor a solução (classificadores hierárquicos)
- Fazer imagem com organização da solução

## 2. Base de Dados

A base de dados utilizada neste trabalho compreende 10 perguntas e respostas para cada um dos 16 primeiros objetivos. Ela foi construída manualmente pelos 4 integrantes do grupo, sendo que, cada integrante focou em 4 objetivos. As perguntas e respostas foram retiradas do relatório das Nações Unidas de 2020 [2]. Portanto, a base de dados tem 160 pares de perguntas e respostas. Para desenvolver um modelo supervisionado vimos que não seria o suficiente, pois cada resposta compreende uma classe distinta. Para resolver esse problema, uma biblioteca chamada NLPAug [3] foi utilizada, a qual tem como objetivo criar mais dados textuais a partir de uma base existente. Com ela foram criados mais 4 registros para cada par fazendo pequenas substituições nas frases, crescendo a base de dados para 800 registros. Essas substituições se referem a utilizar o modelo <i>distilbert</i> para buscar palavras próximas no embedding, e o modelo <i>wornet</i> para buscar sinônimos de palavras das perguntas.

## 3. Desenvolvimento

### 3.1 Sentence embedding

In [ ]:
# Tokenize
tokenized = [word_tokenize(c.lower()) for c in corpus]

# Tag
tagged = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized)]

# Train model
model = Doc2Vec(tagged, vector_size=50, window=2, min_count=1, epochs=100)

### 3.2 Separação dos dados

In [ ]:
# Which subject?
class_sub = {}
X_sub, y_sub = [], []

for i, f in enumerate(files):

    class_sub[str(i+1)] = f
    X_sub.extend(files[f]['question'].values)
    y_sub.extend([str(i+1)]*len(files[f]['question'].values))

In [ ]:
# Which question?
class_que = {}
X_que, y_que = [], []
count = 0
last = ''

for i, f in enumerate(files):
    
    for j, row in files[f].iterrows():
        
        que = row['question']
        ans = row['answer']
        
        if last != ans:
            last = ans
            count += 1
        
        class_que[str(count)] = ans
        X_que.append(que)
        y_que.append(str(count))

## 4. Metodologia

## 5. Resultados

## Conclusões

## Referências

[1] P. Guitarrara, "Desenvolvimento sustentável", acessado em: 17/07/2021. Disponível em: https://brasilescola.uol.com.br/geografia/desenvolvimento-sustentavel.htm.

[2] UN, "The Sustainable Development Goals Report 2020", acessado em: 17/07/2021. Disponível em: https://unstats.un.org/sdgs/report/2020/The-Sustainable-Development-Goals-Report-2020.pdf.

[3] "NLP Augmentation", acessado em: 17/07/2021. Disponível em: https://github.com/makcedward/nlpaug.

## Contribuições
<br>
O membro do grupo <b>Germán Darío Buitrago Salazar</b> contribuiu com...

O membro do grupo <b>Giovanna Relvas Bartilotti</b> contribuiu com...

O membro do grupo <b>Lucas Zanco Ladeira</b> contribuiu com...

O membro do grupo <b>Rafael Scherer</b> contribuiu com...

...........

<b>Observações</b>

A ideia inicial do trabalho era de usar um modelo de transformer (BERT, Roberta, etc), mas tivemos alguns problemas com a implementação de bibliotecas. Pensamos em utilizar SimpleTransformers, Pytorch ou Tensorflow, mas pelo tempo disponível achamos melhor utilizar uma arquitetura mais simples da solução.